In [1]:
import nltk
import numpy as np
#nltk.download("punkt")
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return stemmer.stem(word.lower())

def bag_of_words(tokenize_sentence, all_words):
    tokenize_sentence = [stem(w) for w in tokenize_sentence]
    bag = np.zeros(len(all_words),dtype = np.float32)
    for idx,w in enumerate(all_words):
        if w in tokenize_sentence:
            bag[idx] = 1.0
    return bag
    

In [2]:
#train.py
import json
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
with open("intents.json",'r') as f:
    intents = json.load(f)

all_words = []
tags = []
xy = []

for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w,tag))
        
ignore_words = ['?','!','.',',']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))
print(all_words)
print(tags)

X_train =[]
y_train = []

for (pattern_sentence,tag) in xy:
    bag = bag_of_words(pattern_sentence,all_words)
    X_train.append(bag)
    label = tags.index(tag)
    y_train.append(label)
    
    
X_train = np.array(X_train)
y_train = np.array(y_train)

print(X_train,y_train)

["'s", 'a', 'accept', 'anyon', 'are', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'deliveri', 'do', 'doe', 'funni', 'get', 'good', 'goodby', 'have', 'hello', 'help', 'hey', 'hi', 'how', 'i', 'is', 'item', 'joke', 'kind', 'know', 'later', 'long', 'lot', 'mastercard', 'me', 'my', 'of', 'onli', 'pay', 'paypal', 'see', 'sell', 'ship', 'someth', 'take', 'tell', 'thank', 'that', 'there', 'what', 'when', 'which', 'with', 'you']
['delivery', 'funny', 'goodbye', 'greeting', 'items', 'payments', 'thanks']
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 1.]] [3 3 3 3 3 3 2 2 2 6 6 6 6 4 4 4 5 5 5 5 0 0 0 1 1 1]


In [4]:
import tensorflow as tf
from random import randint

In [5]:
model = tf.keras.models.Sequential()
#model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(300, input_shape = (54,),activation= tf.nn.relu))
model.add(tf.keras.layers.Dense(300, activation= tf.nn.relu))
model.add(tf.keras.layers.Dense(len(tags), activation= tf.nn.softmax))

In [6]:
model.compile(optimizer ='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train,y_train, epochs=50)
X_train[0].shape

Train on 26 samples
Epoch 1/50
26/26 [==============================] - 0s 18ms/sample - loss: 1.9518 - accuracy: 0.1538
Epoch 2/50
26/26 [==============================] - 0s 231us/sample - loss: 1.8622 - accuracy: 0.5000
Epoch 3/50
26/26 [==============================] - 0s 115us/sample - loss: 1.7788 - accuracy: 0.7692
Epoch 4/50
26/26 [==============================] - 0s 115us/sample - loss: 1.6999 - accuracy: 0.9231
Epoch 5/50
26/26 [==============================] - 0s 193us/sample - loss: 1.6245 - accuracy: 0.9231
Epoch 6/50
26/26 [==============================] - 0s 116us/sample - loss: 1.5508 - accuracy: 0.9231
Epoch 7/50
26/26 [==============================] - 0s 154us/sample - loss: 1.4775 - accuracy: 0.9231
Epoch 8/50
26/26 [==============================] - 0s 308us/sample - loss: 1.4038 - accuracy: 0.9615
Epoch 9/50
26/26 [==============================] - 0s 154us/sample - loss: 1.3297 - accuracy: 0.9615
Epoch 10/50
26/26 [==============================] - 0s 115us/s

(54,)

In [7]:
run = True
print("type 'quit' to exit")
while run:
    sentence = input("You :")
    if sentence == 'quit':
        print("Krithi: Byeee")
        break
    #sentence = "tell me a joke"
    sentence = tokenize(sentence)
    sentence = [stem(i) for i in sentence if i not in ignore_words]
    model_bag = bag_of_words(sentence, all_words)
    model_bag = np.array(model_bag)
    model_bag = np.reshape(model_bag,(1,54))
    pre = model.predict(model_bag)
    hi =np.argmax(pre)
    for intent in intents['intents']:
        if intent['tag'] == tags[hi]:
            tag = intent['responses']
            j = randint(0,len(tag)-1)
            k = "Krithi: "+ tag[j]
            print(k)

type 'quit' to exit


You : hi


Krithi: Hi there, what can I do for you?


You : quit


Krithi: Byeee
